# Big Data
# BD01 PySpark  RDD

In [1]:
# Esto solo lo utilizamos para instalar las librerias 
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz       
!tar xf spark-3.0.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
sc = SparkSession.builder.master("local[*]").getOrCreate()

## <font color='blue'>**RDDs**</font>

Los Resilient Distributed Datasets (RDD) son colecciones de objetos JVM inmutables que se distribuyen en un clúster de Apache Spark. 

Un RDD es el tipo de conjunto de datos más fundamental de Apache Spark; cualquier acción en un Spark DataFrame finalmente se traduce en una ejecución altamente optimizada de transformaciones y acciones en RDD. 

Los datos en un RDD se dividen en trozos basados en una clave y luego se dispersan en todos los nodos ejecutores. Los RDD pueden recuperarse rápidamente de cualquier problema, ya que los mismos fragmentos de datos se replican en varios nodos ejecutores. Por lo tanto, incluso si un ejecutor falla, otro seguirá procesando los datos. Esto le permite realizar sus cálculos funcionales contra su conjunto de datos muy rápidamente al aprovechar el poder de múltiples nodos. Los RDD mantienen un registro de todos los pasos de ejecución aplicados a cada fragmento. Esto, además de la replicación de datos, acelera los cálculos y, si algo sale mal, los RDD aún pueden recuperar la parte de los datos perdidos debido a un error del ejecutor.

Si bien es común perder un nodo en entornos distribuidos (por ejemplo, debido a problemas de conectividad, problemas de hardware), la distribución y replicación de los datos protege contra la pérdida de datos, mientras que el linaje de datos permite que el sistema se recupere rápidamente.


**Particions**: Los RDD son una colección de varios datos si no pueden caber en un solo nodo, deben dividirse en varios nodos. Entonces significa que cuanto mayor sea el número de particiones, mayor será el paralelismo. Estas particiones de un RDD se distribuyen por todos los nodos de la red.

**Operaciones con RDD**: Hay dos tipos de operaciones que puede realizar en un RDD: **Transformaciones** y **Acciones**. La transformación aplica alguna función en un RDD y crea un nuevo RDD, no modifica el RDD en el que aplica la función (recuerde que los RDD son inmutables). Además, el nuevo RDD mantiene un puntero a su RDD principal.

Una **acción** se utiliza para guardar el resultado en alguna ubicación o para mostrarlo. También puede imprimir la información del linaje RDD usando el comando

Un RDD puede ser pensado como un conjunto de transformaciones y una acción que collecta el resultado. 

A continuacion mostramos el grafo aciclico dirigido del ciclo de vida de un RDD.
<img src="https://drive.google.com/uc?export=view&id=1nlwjvcpNhFZ0YCJ4aBgl8ioAvBeOdZoi" width=600 height=400/>

**DAGScheduler** es la capa de programación de Apache Spark que implementa la programación orientada por etapas. Transforma un plan de ejecución lógico (es decir, el linaje RDD de dependencias construidas usando transformaciones RDD) en un plan de ejecución físico (usando etapas).

<img src="https://drive.google.com/uc?export=view&id=1WZRvLgBh4IZF0_15St3yT2pfVrDVOWo_" width=600 height=400/>

el  **DAGScheduler** divide el grafo en varias etapas, las etapas se crean en función de las transformaciones. Las transformaciones estrechas se agruparán (en tuberías) juntas en una sola etapa.





### Creemos RDDs

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
myRDD = sc.sparkContext.parallelize( 
 [('Amber', 22), ('Alfred', 23), ('Skye',4), ('Albert', 12), ('Amber', 9)]
)

In [6]:
myRDD.take(2)

[('Amber', 22), ('Alfred', 23)]

El metodo parallelize() crea una collección paralelizada. Esto permite que Spark distribuya los datos en varios nodos, en lugar de depender de un solo nodo para procesar los datos:

<img src="https://drive.google.com/uc?export=view&id=1lcecAAov0cIpEcVcJ9WCbKyANgJqXva-" width=600 height=400/>



## Reading data from files

In [7]:
path ='/content/drive/MyDrive/Cursos/Data Science UDD/Big Data/airport-codes-na.txt'
# myRDD = sc.sparkContext.textFile('airport-codes-na.txt')
myRDD = sc.sparkContext.textFile(path)

In [8]:
myRDD.take(5)

['City\tState\tCountry\tIATA',
 'Abbotsford\tBC\tCanada\tYXX',
 'Aberdeen\tSD\tUSA\tABR',
 'Abilene\tTX\tUSA\tABI',
 'Akron\tOH\tUSA\tCAK']

In [9]:
myRDD.count()

527

In [10]:
myRDD = sc.sparkContext.textFile(path).map(lambda line: line.split("\t"))

In [11]:
myRDD.getNumPartitions()

2

In [12]:
myRDD = sc.sparkContext.textFile(path, minPartitions=4, use_unicode=True).map(lambda line: line.split("\t"))

In [13]:
myRDD.take(5)

[['City', 'State', 'Country', 'IATA'],
 ['Abbotsford', 'BC', 'Canada', 'YXX'],
 ['Aberdeen', 'SD', 'USA', 'ABR'],
 ['Abilene', 'TX', 'USA', 'ABI'],
 ['Akron', 'OH', 'USA', 'CAK']]

In [14]:
myRDD.getNumPartitions()

4

## <font color='green'>**Ejercicio 1**</font>

Lea el data set departuredelays.csv. Con un numero minimo e 8 particiones y realice una transformacion map la cual realice un split de cada  fila. 

Posteriormente imprima 5 elementos y el numero de particiones. 

In [15]:
path ='/content/drive/MyDrive/Cursos/Data Science UDD/Big Data/departuredelays.csv'
myRDD = sc.sparkContext.textFile(path, minPartitions=8)
myRDD.take(5)

['date,delay,distance,origin,destination',
 '01011245,6,602,ABE,ATL',
 '01020600,-8,369,ABE,DTW',
 '01021245,-2,602,ABE,ATL',
 '01020605,-4,602,ABE,ATL']

In [16]:
myRDD = myRDD.map(lambda line: line.split(','))

In [17]:
myRDD.take(5)

[['date', 'delay', 'distance', 'origin', 'destination'],
 ['01011245', '6', '602', 'ABE', 'ATL'],
 ['01020600', '-8', '369', 'ABE', 'DTW'],
 ['01021245', '-2', '602', 'ABE', 'ATL'],
 ['01020605', '-4', '602', 'ABE', 'ATL']]

In [18]:
myRDD.getNumPartitions()

8

<font color='green'>**Fin Ejercicio 1**</font>

#### *Usando DataFrame*
Tenga en cuenta que es más rápido (2,44 s para DF, 2,96 s para RDD con 8 particiones) mientras que DF también tiene en cuenta el encabezado y puede inferir el esquema

In [19]:
path = "/content/drive/MyDrive/Cursos/Data Science UDD/Big Data/"
myDF = sc.read.csv(path + 'departuredelays.csv', header=True, inferSchema=True)
myDF.count()

1391578

In [20]:
myDF.show()

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|    6|     602|   ABE|        ATL|
|1020600|   -8|     369|   ABE|        DTW|
|1021245|   -2|     602|   ABE|        ATL|
|1020605|   -4|     602|   ABE|        ATL|
|1031245|   -4|     602|   ABE|        ATL|
|1030605|    0|     602|   ABE|        ATL|
|1041243|   10|     602|   ABE|        ATL|
|1040605|   28|     602|   ABE|        ATL|
|1051245|   88|     602|   ABE|        ATL|
|1050605|    9|     602|   ABE|        ATL|
|1061215|   -6|     602|   ABE|        ATL|
|1061725|   69|     602|   ABE|        ATL|
|1061230|    0|     369|   ABE|        DTW|
|1060625|   -3|     602|   ABE|        ATL|
|1070600|    0|     369|   ABE|        DTW|
|1071725|    0|     602|   ABE|        ATL|
|1071230|    0|     369|   ABE|        DTW|
|1070625|    0|     602|   ABE|        ATL|
|1071219|    0|     569|   ABE|        ORD|
|1080600|    0|     369|   ABE| 

In [21]:
myDF.rdd.getNumPartitions()

2

In [22]:
myDF.printSchema()

root
 |-- date: integer (nullable = true)
 |-- delay: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)



## <font color='blue'>**RDD Transformations**</font>

Una vez creado un RDD, es frecuente usar el método take() para devolver los valores a la consola o notebook. take() es una **acción** RDD. Tenga en cuenta que un enfoque común en PySpark es usar collect(), que devuelve todos los valores en su RDD desde los nodos de trabajo de Spark al controlador. Existen implicaciones de rendimiento cuando se trabaja con una gran cantidad de datos, ya que esto se traduce en grandes volúmenes de datos que se transfieren desde los nodos de trabajo de Spark al controlador. Para pequeñas cantidades de datos (como aquí), esto está perfectamente bien, pero, como una cuestión de costumbre, casi siempre debería usar el método take(n) en su lugar; devuelve los primeros n elementos del RDD en lugar de todo el conjunto de datos. Es un método más eficiente porque primero escanea una partición y usa esas estadísticas para determinar el número de particiones necesarias para devolver los resultados.
<img src="https://drive.google.com/uc?export=view&id=1QWs2K13TW0wT2HK1h-4AkaOzKCFd3pFD" width=800 height=400/>



#### Getting Ready

In [23]:
airports = sc.sparkContext.textFile(path + 'airport-codes-na.txt').map(lambda line: line.split("\t"))
airports.take(5)

[['City', 'State', 'Country', 'IATA'],
 ['Abbotsford', 'BC', 'Canada', 'YXX'],
 ['Aberdeen', 'SD', 'USA', 'ABR'],
 ['Abilene', 'TX', 'USA', 'ABI'],
 ['Akron', 'OH', 'USA', 'CAK']]

In [24]:
flights = sc.sparkContext.textFile(path + 'departuredelays.csv').map(lambda line: line.split(","))
flights.take(5)

[['date', 'delay', 'distance', 'origin', 'destination'],
 ['01011245', '6', '602', 'ABE', 'ATL'],
 ['01020600', '-8', '369', 'ABE', 'DTW'],
 ['01021245', '-2', '602', 'ABE', 'ATL'],
 ['01020605', '-4', '602', 'ABE', 'ATL']]

### map()

Los componentes clave de esta transformación de map son:

1. lambda: una función anónima (es decir, una función definida sin un
nombre) compuesto por una sola expresión
2. split: Estamos usando la función split de PySpark (dentro de pyspark.sql.functions) para dividir una cadena alrededor de un patrón de expresión regular; en este caso, nuestro delimitador es una pestaña (es decir, \ t)

In [25]:
airports.map(lambda c: (c[0], c[1])).take(5)

[('City', 'State'),
 ('Abbotsford', 'BC'),
 ('Aberdeen', 'SD'),
 ('Abilene', 'TX'),
 ('Akron', 'OH')]

### filter()

La transformación de filter(f) devuelve un nuevo RDD basado en la selección de elementos para lo cual la función f devuelve verdadero.

In [26]:
airports.map(lambda c: (c[0], c[1])).filter(lambda c: c[1] == "WA").take(5)

[('Bellingham', 'WA'),
 ('Moses Lake', 'WA'),
 ('Pasco', 'WA'),
 ('Pullman', 'WA'),
 ('Seattle', 'WA')]

### flatMap()

La transformación flatMap (f) es similar a map, pero el nuevo RDD se aplana
todos los elementos (es decir, una secuencia de eventos).

In [27]:
airports.filter(lambda c: c[1] == "WA").map(lambda c: (c[0], c[1])).flatMap(lambda x: x).take(10)

['Bellingham',
 'WA',
 'Moses Lake',
 'WA',
 'Pasco',
 'WA',
 'Pullman',
 'WA',
 'Seattle',
 'WA']

### distinct()

La transformación distinct() devuelve un nuevo RDD que contiene los distintos
elementos del RDD de origen.

In [28]:
airports.map(lambda c: c[2]).distinct().take(5)

['Country', 'USA', 'Canada']

### sample()

La transformación de sample(withReplacement, fraction, seed) muestrea una fracción de los datos, con o sin reemplazo (el parámetro withReplacement), basándose en una semilla aleatoria.

In [29]:
flights.map(lambda c: c[3]).sample(False, 0.001, 123).take(5)

['ABQ', 'AEX', 'AGS', 'ANC', 'ATL']

### join()

https://spark.apache.org/docs/latest/sql-ref-syntax-qry-select-join.html

In [30]:
flights.map(lambda c: (c[3], c[0])).take(5)

[('origin', 'date'),
 ('ABE', '01011245'),
 ('ABE', '01020600'),
 ('ABE', '01021245'),
 ('ABE', '01020605')]

In [31]:
flights.take(5)

[['date', 'delay', 'distance', 'origin', 'destination'],
 ['01011245', '6', '602', 'ABE', 'ATL'],
 ['01020600', '-8', '369', 'ABE', 'DTW'],
 ['01021245', '-2', '602', 'ABE', 'ATL'],
 ['01020605', '-4', '602', 'ABE', 'ATL']]

In [32]:
airports.map(lambda c: (c[3], c[1])).take(5)

[('IATA', 'State'), ('YXX', 'BC'), ('ABR', 'SD'), ('ABI', 'TX'), ('CAK', 'OH')]

In [33]:
flt = flights.map(lambda c: (c[3], c[0]))
air = airports.map(lambda c: (c[3], c[1]))
flt.join(air).take(20)

[('ABE', ('01011245', 'PA')),
 ('ABE', ('01020600', 'PA')),
 ('ABE', ('01021245', 'PA')),
 ('ABE', ('01020605', 'PA')),
 ('ABE', ('01031245', 'PA')),
 ('ABE', ('01030605', 'PA')),
 ('ABE', ('01041243', 'PA')),
 ('ABE', ('01040605', 'PA')),
 ('ABE', ('01051245', 'PA')),
 ('ABE', ('01050605', 'PA')),
 ('ABE', ('01061215', 'PA')),
 ('ABE', ('01061725', 'PA')),
 ('ABE', ('01061230', 'PA')),
 ('ABE', ('01060625', 'PA')),
 ('ABE', ('01070600', 'PA')),
 ('ABE', ('01071725', 'PA')),
 ('ABE', ('01071230', 'PA')),
 ('ABE', ('01070625', 'PA')),
 ('ABE', ('01071219', 'PA')),
 ('ABE', ('01080600', 'PA'))]

In [34]:
flt = flights.map(lambda c: (c[3], c[0]))
air = airports.map(lambda c: (c[3], c[1]))
flt.join(air)

PythonRDD[77] at RDD at PythonRDD.scala:53

In [35]:
flt

PythonRDD[78] at RDD at PythonRDD.scala:53

### repartition()

La transformación de repartition(n) reparte el RDD en n particiones mediante la reorganización aleatoria y la distribución uniforme de los datos a través de la red. Esto puede mejorar el rendimiento al ejecutar más subprocesos paralelos al mismo tiempo

In [36]:
flights.getNumPartitions()

2

In [37]:
flights2 = flights.repartition(8)
flights2.getNumPartitions()

8

In [38]:
rdd = sc.sparkContext.parallelize([1, 2, 3, 4], 4)
def f(splitIndex, iterator): yield splitIndex
rdd.mapPartitionsWithIndex(f).sum()

6

## <font color='green'>**Ejercicio 2**</font>

Otro tipo e datos interesante en spark corresponde al dataframe. 

1. utilce en sc, el metodo read.json para leer el archivo people.json. 
2. Posteriormente muestre los datos. 
3. Visualice el esquema con printSchema
4. Realice un describe
5. Cree una nueva columna que se llame 3x age en la cual se almacena el valor de la edad multiplicado por 3. 



In [39]:
path

'/content/drive/MyDrive/Cursos/Data Science UDD/Big Data/'

In [40]:
json = sc.read.json(path + "people.json")

In [41]:
json.show()

+----+--------+
| age|    name|
+----+--------+
|null| Michael|
|  30|    Andy|
|  19|  Justin|
|  11|     Ana|
|  44|Patricia|
|  89|     Leo|
+----+--------+



In [42]:
json.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [43]:
json.describe()

DataFrame[summary: string, age: string, name: string]

In [45]:
json.withColumn("3x age", json.age * 3).show()

+----+--------+------+
| age|    name|3x age|
+----+--------+------+
|null| Michael|  null|
|  30|    Andy|    90|
|  19|  Justin|    57|
|  11|     Ana|    33|
|  44|Patricia|   132|
|  89|     Leo|   267|
+----+--------+------+



In [46]:
json.withColumn("3x age", json.age * 3).dropna().show()

+---+--------+------+
|age|    name|3x age|
+---+--------+------+
| 30|    Andy|    90|
| 19|  Justin|    57|
| 11|     Ana|    33|
| 44|Patricia|   132|
| 89|     Leo|   267|
+---+--------+------+



In [47]:
json.withColumn("3x age", json.age * 3).dropna().filter(json.name != "Andy").show()

+---+--------+------+
|age|    name|3x age|
+---+--------+------+
| 19|  Justin|    57|
| 11|     Ana|    33|
| 44|Patricia|   132|
| 89|     Leo|   267|
+---+--------+------+



<font color='green'>**Fin ejercicio 2**</font>

## <font color='blue'>**RDD Actions**</font>

Same Getting Ready as Transformations

In [ ]:
# take(n)
airports.take(3)

[['City', 'State', 'Country', 'IATA'],
 ['Abbotsford', 'BC', 'Canada', 'YXX'],
 ['Aberdeen', 'SD', 'USA', 'ABR']]

In [ ]:
# collect()
airports.filter(lambda c: c[1] == "WA").collect()

[['Bellingham', 'WA', 'USA', 'BLI'],
 ['Moses Lake', 'WA', 'USA', 'MWH'],
 ['Pasco', 'WA', 'USA', 'PSC'],
 ['Pullman', 'WA', 'USA', 'PUW'],
 ['Seattle', 'WA', 'USA', 'SEA'],
 ['Spokane', 'WA', 'USA', 'GEG'],
 ['Walla Walla', 'WA', 'USA', 'ALW'],
 ['Wenatchee', 'WA', 'USA', 'EAT'],
 ['Yakima', 'WA', 'USA', 'YKM']]

In [ ]:
# reduce(f)
flights\
   .filter(lambda c: c[3] == 'SEA' and c[4] == 'SFO')\
   .map(lambda c: int(c[1]))\
   .reduce(lambda x, y: x + y)

22293

In [ ]:
flights.take(5)

[['date', 'delay', 'distance', 'origin', 'destination'],
 ['01011245', '6', '602', 'ABE', 'ATL'],
 ['01020600', '-8', '369', 'ABE', 'DTW'],
 ['01021245', '-2', '602', 'ABE', 'ATL'],
 ['01020605', '-4', '602', 'ABE', 'ATL']]

In [ ]:
airports.filter(lambda c: c[1] == "WA").count()

9

In [ ]:
# count
airports.filter(lambda c: c[1] == "WA").count()

9

In [ ]:
# saveAsTextFile
airports.saveAsTextFile("/tmp/denny/airports")

## <font color='green'>**Ejercicio 3**</font>

**Estime el valor de $\pi$ utilizando spark:**

Este cálculo se basa en la siguiente heurística: Por definición, π es el área A Círculo de un círculo con radio r = 1 (generalmente, $πr^2$ es el área de un círculo de radio r).

Luego, se circunscribe este círculo unitario con un cuadrado cuya área es igual a $A_{Square} = 4$. La razón de estas dos áreas equivale a $\frac{A_{Circle}}{A_{Square}} = \frac{π}{4}$ y da la probabilidad geométrica de que un punto dentro del cuadrado se encuentre dentro del círculo.

Supongamos ahora que elegimos un gran número $n$ de puntos al azar dentro del cuadrado circunscrito, por ejemplo, lanzando dardos o dejando caer gotas de lluvia sobre él. Un cierto número $n_{in}$ de estos puntos terminará dentro del área descrita por el círculo, mientras que el número restante $n_{out}$ de estos puntos quedará fuera de él (pero dentro del cuadrado). Por lo tanto, $n_{in} + n_{out} = n$  y la probabilidad de que un punto se encuentre dentro del área del círculo es $\frac{n_{in}}{n}$.

Entonces heuristicamente uno tiene $\frac{A_{Circle}}{A_{Square}} \approx \frac{n_{in}}{n}$ con lo cual podemos estimar pi.

In [59]:
from time import time
import numpy as np
from random import random
from operator import add

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Pi').getOrCreate()
sc_ = spark.sparkContext

n = 10000000


def is_point_inside_unit_circle(p):
    x, y = random(), random()
    return 1 if x*x + y*y < 1 else 0


t_0 = time()

count = sc_.parallelize(range(0, n)) \
             .map(is_point_inside_unit_circle).reduce(add)
print(np.round(time()-t_0, 3), "seconds elapsed for spark approach and n=", n)
print("Pi is roughly %f" % (4.0 * count / n))

spark.stop()

5.071 seconds elapsed for spark approach and n= 10000000
Pi is roughly 3.142005


<font color='green'>**Fin Ejercicio 3**</font>

## <font color='green'>**Ejercicio 4**</font>

Construya con Spark una rutina que permita realizar el conteo de palabras.

1. Lea un archivo text con spark.read.text
2. Realice un map de cada una de las lines leidas rdd.map(lambda r: r[0])
3. Para cada linea, realice un flatMap, dpnde se realiza un split por espacio.
4. Luego volvemos a mapear cada palabra a la tupla (palabra,1)
5 Realice un reducedByKey(add) para contar las repeticiones de cada palaba.
6. Finalmente realice un collect e imprima los resultados palabra, cantidad.

In [ ]:
path

'/content/drive/MyDrive/Cursos/Data Science UDD/Big Data/'

In [ ]:
texto = sc.read.text(path + 'Texto.txt')

In [ ]:
texto.show()

+--------------------+
|               value|
+--------------------+
|Las redes neurona...|
+--------------------+



In [ ]:
txt = texto.rdd.map(lambda c: (c[0])).flatMap(lambda x: x.split(' '))

In [ ]:
txt.take(5)

['Las', 'redes', 'neuronales', 'artificiales', '(también']

In [ ]:
rdd_ = txt.map(lambda x: (x, 1))

In [ ]:
rdd2 = rdd_.reduceByKey(lambda x, y: x + y)

In [ ]:
rdd2.take(5)

[('Las', 2),
 ('redes', 3),
 ('neuronales', 3),
 ('artificiales', 1),
 ('(también', 1)]

In [ ]:
for word in rdd2.collect():
    print(word)

('Las', 2)
('redes', 3)
('neuronales', 3)
('artificiales', 1)
('(también', 1)
('conocidas', 1)
('como', 2)
('sistemas', 2)
('conexionistas)', 1)
('son', 2)
('un', 4)
('modelo', 1)
('computacional', 1)
('el', 6)
('que', 6)
('fue', 1)
('evolucionando', 1)
('a', 7)
('partir', 1)
('de', 25)
('diversas', 2)
('aportaciones', 1)
('científicas', 1)
('están', 1)
('registradas', 1)
('en', 6)
('la', 12)
('historia.1\u200b', 1)
('Consiste', 1)
('conjunto', 1)
('unidades,', 1)
('llamadas', 1)
('neuronas', 3)
('artificiales,', 1)
('conectadas', 1)
('entre', 1)
('sí', 2)
('para', 1)
('transmitirse', 1)
('señales.', 1)
('La', 1)
('información', 1)
('entrada', 1)
('atraviesa', 1)
('red', 3)
('neuronal', 2)
('(donde', 1)
('se', 7)
('somete', 1)
('operaciones)', 1)
('produciendo', 1)
('unos', 4)
('valores', 2)
('salida.', 1)
('Cada', 1)
('neurona', 2)
('está', 1)
('conectada', 1)
('con', 2)
('otras', 1)
('través', 1)
('enlaces.', 1)
('En', 1)
('estos', 1)
('enlaces', 2)
('valor', 4)
('salida', 2)
('anter

<font color='green'>**Fin ejercicio 4**</font>